# Случайные процессы. Прикладной поток.
## Практическое задание 2

**Правила:**

* Выполненную работу нужно отправить на почту `probability.diht@yandex.ru`, указав тему письма `"[СП17] Фамилия Имя - Задание 2"`. Квадратные скобки обязательны. Вместо `Фамилия Имя` нужно подставить свои фамилию и имя.
* Прислать нужно ноутбук и его pdf-версию. Названия файлов должны быть такими: `2.N.ipynb` и `2.N.pdf`, где `N` - ваш номер из таблицы с оценками.
* Никакой код из данного задания при проверке запускаться не будет.

In [1]:
import numpy as np
import scipy.stats as sps
from collections import Counter  # это может пригодиться
from BranchingProcess import Person, BranchingProcess, read_from_files

from statsmodels.sandbox.stats.multicomp import multipletests
from math import factorial, exp

import matplotlib.pyplot as plt
from matplotlib import rcParams
rcParams.update({'font.size': 16})
%matplotlib inline

Считаем данные.

In [2]:
processes = read_from_files(["data/D.txt", "data/W.txt", "data/S.txt", 
                             "data/I.txt", "data/N.txt", "data/O.txt", 
                             "data/K.txt", "data/J.txt", "data/M.txt", 
                             "data/G.txt"])
print(len(processes))

76628


В имеющихся данных очень много людей, про которых известно лишь то, что они когда-то существовали. Обычно их фамилия неизвестна (вместо фамилии у них может стоять, к примеру, `B-290`), а у некоторых из них неизвестен даже пол, не говоря уже о родителях и детях. Такие данные стоит удалить.

Удалим все процессы, состоящие только из одного поколения (в котором, естественно, будет только один человек).

In [3]:
for i in range(len(processes))[::-1]:
    if len(processes[i].generations) < 2:
        del processes[i]

print(len(processes))

22166


## Оценка закона размножения

Чтобы проводить какой-либо анализ ветвящегося процесса нужно некоторым образом оценить закон размножения. Кажется,  что для этого достаточно посчитать количество сыновей у каждого человека, получив тем самым выборку неотрицательных целых чисел. Однако, проблема в том, что данные неполные, в частности, некоторые поля могут быть не заполнены. Тем не менее обычно у человека указаны либо все дети, либо не указаны вообще. 
Таким образом, условно мы можем разделить выборку на две части: поле детей заполнено (в т.ч. если у человека на самом деле нет детей), поле детей незаполнено. Если бы первая часть выборки была бы полностью известна, что распределение можно оценить по ней. Нам же неизвестен размер выборки и количество нулевых элементов в ней. Количество положительных элементов известно.


**Математическая постановка задачи**

$\mathsf{P}_\theta$ --- неизвестное распределение из некоторого класса распределений $\mathcal{P}$ на $\mathbb{Z}_+$.

$X_1, ..., X_n$ --- выборка из распределения $\mathsf{P}_\theta$, причем $n$ и количество нулей в выборке неизвестны. 

$Y_1, ..., Y_s$ --- положительная подвыборка, которая полностью нам известна. В нашей задаче $Y_j$ --- количество сыновей у $j$-го человека среди тех, у кого есть хотя бы один сын.

Оценку параметра $\theta$ можно найти методом максимального правдоподобия:

$$\prod_{i=1}^s \mathsf{P}_\theta (Y_i \left| Y_i > 0 \right) \to \max_\theta$$ 

** Геометрическое распределение **

Функция вероятности геометрического распределения есть

$$p_{\theta}(k) = p(1 - p)^{k},$$

где $k$ - номер первого успеха.

Так как рассматриваются только положительные значения, то будем работать с функцией

$$p_{\theta}(k\ |\ k>0) = p(1-p)^{k-1}$$

Тогда необходимо максимизировать функцию 

$$f = \cfrac{p^{s}(1 - p)^{\sum_{i=1}^{s}Y_{i}}}{(1 - p)^s}$$

Прологарифмировав и взяв производную, получаем

$$L = \cfrac{s}{p} - \cfrac{-s + \sum_{i=1}^{s}Y_{i}}{1 - p}$$

Таким образом, оценкой параметра $\theta$ является $\theta^* = \cfrac{1}{\overline{Y}}$.

In [56]:
# Генерация выборки
# Смотрим на мужчин в поколении i и проверяем их детей в поколении i + 1

sample = []

for pedigree in processes:
    for i in range(len(pedigree.generations) - 1):
        for person in pedigree.generations[i]:
            if person.gender == "male" and (len(person.children) > 0):
                tmp = 0
                for child in pedigree.generations[i+1]:
                    if child.gender == "male" and \
                    (child.name in person.children):
                        tmp += 1
                if (tmp != 0):
                    sample.append(tmp)

sample = np.array(sample)
theta_geom = 1 / np.mean(sample)

In [57]:
print('Оценка параметра для геометрического распределения: ', round(theta_geom, 2))

Оценка параметра для геометрического распределения:  0.46


** Первая часть **

Для того, чтобы найти вероятность вырождения, режим уравнение

$$q = \varphi_{\xi}(q),$$

где $\varphi$ - производящая функция.

Для геометрического распределения имеем

$$ \varphi_{\xi}(z) = \mathbf{E}z^{\xi} = \sum_{m=1}^{\infty} P_{m}z^{m} = \sum_{m=1}^{\infty}(p(1-p)^{m})z^{m} = p \sum_{m=1}^{\infty}((1-p)z)^{m} = \cfrac{p}{1-(1-p)z}$$

$$q = \cfrac{p}{1-(1-p)q} \Longrightarrow q = \cfrac{p}{1-p}I_{\{p\leq\frac{1}{2}\}} + I_{\{p>\frac{1}{2}\}}$$

In [58]:
# Функция, вычисляющая вероятность вырождения

def count_dgnrcy(p):
    if (p > 0.5):
        return 1
    else:
        return p / (1 - p)

In [59]:
# Генерация выборки

# atomic - массив, элементами которого являются массивы. 
# atomic[i] - массив количества детей в i-м роде

# sample - массив с уже рассчитанными вероятностями 
# вырождения

degeneracy = []

for pedigree in processes:
    atomic = []
    
    for i in range(len(pedigree.generations) - 1):
        for person in pedigree.generations[i]:
            if person.gender == "male" and (len(person.children) > 0):
                tmp = 0
                for child in pedigree.generations[i+1]:
                    if child.gender == "male" and \
                    (child.name in person.children):
                        tmp += 1
                if (tmp != 0):
                    atomic.append(tmp)
                    
    # Согласно условию, если в роду нет мужчин, то вероятность
    # вырождения полагается равной нулю. Если их меньше 10, то
    # значению, полученному для общей выборки.
    # Иначе - вычисляется явно.    
    
    if len(atomic) == 0:
        degeneracy.append(1)
    elif len(atomic) < 10:
        degeneracy.append(count_dgnrcy(theta_geom))
    else:
        degeneracy.append(count_dgnrcy(1 / np.mean(atomic)))

sample = np.array(sample)

In [61]:
print("Количество процессов, вероятность вырождения которых равна 1: ",
      sum(i==1 for i in degeneracy))
print("Количество процессов, вероятность вырождения которых меньше 0.5: ",
      sum(i<0.5 for i in degeneracy))

Количество процессов, вероятность вырождения которых равна 1:  7335
Количество процессов, вероятность вырождения которых меньше 0.5:  169


** Вторая часть **